In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, vocab_size=100, embed_dim=128):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)

        self.conv1 = nn.Conv1d(embed_dim, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 128, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(128 * 2, 128)
        self.fc2 = nn.Linear(128, 1)

    def encode(self, x):
        x = self.embedding(x)        # (batch, seq_len, embed)
        x = x.permute(0, 2, 1)       # (batch, embed, seq_len)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.mean(x, dim=2)     # Global Avg Pool
        return x

    def forward(self, drug, target):
        drug_feat = self.encode(drug)
        target_feat = self.encode(target)
        x = torch.cat([drug_feat, target_feat], dim=1)
        x = F.relu(self.fc1(x))
        return self.fc2(x).squeeze()

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
cnn_model = CNNModel(vocab_size=100).to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()
epochs = 3

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
class DTADataset(Dataset):
    def __init__(self, drugs, targets, labels):
        self.drugs = drugs
        self.targets = targets
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (
            self.drugs[idx],
            self.targets[idx],
            self.labels[idx]
        )


In [ ]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

N = 500
seq_len = 50
vocab_size = 100

drug_tensor = torch.randint(0, vocab_size, (N, seq_len))
target_tensor = torch.randint(0, vocab_size, (N, seq_len))
y_tensor = torch.rand(N)


In [ ]:
X_drug_train, X_drug_test, \
X_target_train, X_target_test, \
y_train, y_test = train_test_split(
    drug_tensor,
    target_tensor,
    y_tensor,
    test_size=0.2,
    random_state=42
)

In [ ]:
class DTADataset(Dataset):
    def __init__(self, drugs, targets, labels):
        self.drugs = drugs
        self.targets = targets
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (
            self.drugs[idx],
            self.targets[idx],
            self.labels[idx]
        )

train_dataset = DTADataset(X_drug_train, X_target_train, y_train)
test_dataset  = DTADataset(X_drug_test,  X_target_test,  y_test)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, vocab_size=100, embed_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv1 = nn.Conv1d(embed_dim, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 1)

    def encode(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return torch.mean(x, dim=2)

    def forward(self, drug, target):
        d = self.encode(drug)
        t = self.encode(target)
        x = torch.cat([d, t], dim=1)
        x = F.relu(self.fc1(x))
        return self.fc2(x).squeeze()


In [ ]:
cnn_model = CNNModel(vocab_size=100).to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

epochs = 3
for epoch in range(epochs):
    cnn_model.train()
    total_loss = 0

    for drug, target, y in train_loader:
        drug, target, y = drug.to(device), target.to(device), y.to(device)

        optimizer.zero_grad()
        pred = cnn_model(drug, target)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

cnn_model.eval()
preds, targets = [], []

with torch.no_grad():
    for drug, target, y in test_loader:
        drug, target = drug.to(device), target.to(device)
        pred = cnn_model(drug, target)
        preds.extend(pred.cpu().numpy())
        targets.extend(y.numpy())

print("MSE:", mean_squared_error(targets, preds))
print("MAE:", mean_absolute_error(targets, preds))

plt.scatter(targets, preds, alpha=0.5)
plt.xlabel("True Binding")
plt.ylabel("Predicted Binding")
plt.title("CNN Results")
plt.show()


In [ ]:
print("\nFirst 10 predictions:")
for i in range(10):
    print(
        f"Sample {i+1}: "
        f"True = {targets[i]:.4f} | "
        f"Predicted = {preds[i]:.4f}"
    )
